<a href="https://colab.research.google.com/github/divya2648/CS682/blob/main/CS682_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 17.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=e8ad3ad67a68e12faa32db5952d6d11d3007bd6191b4f27132563168789513a2
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [ ]:
import pandas as pd
import requests
import json
import csv
from geopy import distance
from geopy.geocoders import Nominatim
from google.colab import drive
#Authorize access to google drive
drive.mount('/content/drive', force_remount=True)

import plotly.express as px
import plotly.graph_objects as go

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt

Mounted at /content/drive


In [ ]:
#Dictionary to map US states to State Codes
us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC",
"American Samoa": "AS",
"Guam": "GU",
"Northern Mariana Islands": "MP",
"Puerto Rico": "PR",
"United States Minor Outlying Islands": "UM",
"U.S. Virgin Islands": "VI",
}

# **Fetch irradiance and temperature for given citites**

In [ ]:
# Fetch the lat-lon file into a dataframe
dfll = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/lat-lon.xlsx')

# Add two new columns "City Name" and "State Code/Country Name"
dfll["City Name"] = pd.NaT
dfll["State Code/Country Name"] = pd.NaT

# Use geolocator to fetch city and state code (for US cities)/ country name using latitude and longitude and populate dfll dataframe
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfll.iterrows():
  address = geolocator.reverse(str(row[0])+","+str(row[1])).raw['address']
  dfll.loc[index,"City Name"]=address.get('city', '')
  if address.get('country', '')== "United States":
    dfll.loc[index,"State Code/Country Name"]= us_state_to_abbrev.get(address.get('state', ''))
  else:
    dfll.loc[index,"State Code/Country Name"]= address.get('country', '')

#Drop NaT values and update the lat-lon file
dfll = dfll.dropna()

In [ ]:
#fetch the city-name file into a dataframe
dfcity = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/city-name.xlsx')

#Add two new columns "Latitude" and "Longitude"
dfcity["Latitude"] = pd.NaT
dfcity["Longitude"] = pd.NaT

# Use geolocator to latitude and longitude using city and state code (for US cities)/ country name and populate dfcity dataframe
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfcity.iterrows():
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    dfcity.loc[index,"Latitude"]=location.latitude
    dfcity.loc[index,"Longitude"]=location.longitude

#Drop NaT values and update the lat-lon file
dfcity=dfcity.dropna()

In [ ]:
#Concatenate lat-lon and city-name dataframes without duplicates
df_irradiance_temp = pd.concat([dfcity,dfll], axis = 0).drop_duplicates().reset_index(drop=True)

In [ ]:
#Add four new columns "Irradiance", "Avg temp", "Max temp", and "Min temp"
df_irradiance_temp["Irradiance"] = pd.NaT
df_irradiance_temp["Avg temp"]= pd.NaT
df_irradiance_temp["Max temp"]= pd.NaT
df_irradiance_temp["Min temp"]= pd.NaT
for index,row in df_irradiance_temp.iterrows():
  #Get irradiance and temperature for all the latitude and longitude values in the dataframe
  json_data = json.loads(requests.get('https://power.larc.nasa.gov/api/temporal/monthly/point?start=2016&end=2020&latitude='+str(row[2])+'&longitude='+str(row[3])+'&community=re&parameters=ALLSKY_SFC_SW_DWN%2CT2M%2CT2M_MAX%2CT2M_MIN&format=json&header=true').text)
  irradiance = json_data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
  AvgTemp = json_data['properties']['parameter']['T2M']
  maxTemp = json_data['properties']['parameter']['T2M_MAX']
  minTemp = json_data['properties']['parameter']['T2M_MIN']
  df_irradiance_temp.loc[index,"Irradiance"] = (irradiance['201613']+irradiance['201713']+irradiance['201813']+irradiance['201913']+irradiance['202013'])/5
  df_irradiance_temp.loc[index,"Avg temp"] = (AvgTemp['201613']+AvgTemp['201713']+AvgTemp['201813']+AvgTemp['201913']+AvgTemp['202013'])/5
  df_irradiance_temp.loc[index,"Max temp"] = (maxTemp['201613']+maxTemp['201713']+maxTemp['201813']+maxTemp['201913']+maxTemp['202013'])/5
  df_irradiance_temp.loc[index,"Min temp"] = (minTemp['201613']+minTemp['201713']+minTemp['201813']+minTemp['201913']+minTemp['202013'])/5

In [ ]:
#Creating a new irradiance-temp.xlsx in the output folder
df_irradiance_temp.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/irradiance-temp.xlsx',index=False)

# **Fetch Average NO2 for given cities**

In [ ]:
# Get the contents of NASA Timeseries files through URL
!wget https://so2.gsfc.nasa.gov/no2/pix/time_series/OMNO2_Timeseries_AllCities.csv -q
data=pd.read_csv('OMNO2_Timeseries_AllCities.csv',index_col=False, on_bad_lines='skip',skiprows=7)
df_average_NO2 = pd.DataFrame(data)

In [ ]:
sum_med = df_average_NO2.groupby('City')['Median'].mean()

with open('OMNO2_Timeseries_AllCities.csv','r') as f_in:
    f = csv.reader(f_in)
    rows=df_average_NO2.groupby(['City','Country','LonCenter','LatCenter']).agg({'Median': ['mean']})
    rows.columns = ['Average NO2']
    rows = rows.reset_index()

for index, row in rows.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    rows.loc[index,"LatCenter"]=location.latitude
    rows.loc[index,"LonCenter"]=location.longitude

nasa_cities = rows.rename(columns={'City': 'City Name','LonCenter':'Longitude','LatCenter':'Latitude'})

City
Aberdeen     -0.214377
Abidjan      -0.276345
AbuDhabi      2.567037
Abuja         1.008219
Accra        -1.260332
               ...    
Yangoon     -13.507840
Yaounde      -0.735572
Yerevan       1.204523
Zhengzhou     6.623809
Zurich        1.627309
Name: Median, Length: 316, dtype: float64


In [ ]:
nasa_cities = nasa_cities.apply(pd.to_numeric, errors='ignore')
df_irradiance_temp = df_irradiance_temp.apply(pd.to_numeric, errors='ignore')
df_irradiance_temp_NO2 = df_irradiance_temp
#NO2data = pd.merge(customer_cities,nasa_cities , on=['Longitude','Latitude'])
df_irradiance_temp_NO2["Average NO2"] = pd.NaT

# Check the nearest city based on latitude and longitude values
for index, row in df_irradiance_temp_NO2.iterrows():
  min_distance=float('inf')
  for nindex, nrow in nasa_cities.iterrows():
    if(distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles < min_distance):
      min_distance = distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles
      min_index = nindex
  df_irradiance_temp_NO2.loc[index,'Average NO2'] = nasa_cities['Average NO2'].values[min_index]

In [ ]:
#Creating a new Average NO2 data in the output folder
df_irradiance_temp_NO2.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/average_no2_data.xlsx',index=False)

# **Bubble maps of Irradiance data:**

In [ ]:
df1 = df_irradiance_temp_NO2

#Convert the "object" type columns to numeric to facilitate plotting
df1 = df1.apply(pd.to_numeric, errors='ignore')

#Data to be visible on hover
df1['text'] = 'City Name: '+df1["City Name"]+ \
              '<br>State Code/Country Name: '+df1["State Code/Country Name"]+ \
              '<br>Irradiance: '+df1["Irradiance"].astype(str)+ \
              '<br>Avg temp: '+df1["Avg temp"].astype(str)+ \
              '<br>Max temp: '+df1["Max temp"].astype(str)+ \
              '<br>Min temp: '+df1["Min temp"].astype(str)+ \
              '<br>Average NO2: '+df1["Average NO2"].astype(str)
names = ["0%-10%","10%-20%","20%+"]

In [ ]:
#Bubble map of the world against Irradiance data:

# Sort values on Irradiance to obtain the top 10%, 10%-20% and 20%+
df1 = df1.sort_values(['Irradiance'], ascending=[0],ignore_index=True)
for index,row in df1.iterrows():
  if index+1 <= int(len(df1.index)*10/100):
    df1.loc[index,'colors'] = "10%"
  elif index+1 <= int(len(df1.index)*10/100)+int(len(df1.index)*20/100):
    df1.loc[index,'colors'] = "10-20%"
  else:
    df1.loc[index,'colors'] = "20%+"
  
# Plot the bubblemap of the natural earth with Irradiance data
fig = px.scatter_geo(df1, lat=df1.Latitude,
                     lon=df1.Longitude,
                     color_discrete_sequence=[ "red", "orange", "green"],
                     color = df1['colors'],
                     hover_data=["City Name", "State Code/Country Name", "Latitude", "Longitude", "Irradiance","Avg temp","Max temp","Min temp","Average NO2"], 
                     size=df1['Irradiance'],
                     projection="natural earth")
fig.show()

In [ ]:
# Bubble map of the USA against Irradiance data:

# Set values for the sorted Irradiance to obtain the top 10%, 10%-20% and 20%+ and colorcode accordingly
limits = [(0,int(len(df1.index)*10/100)),(int(len(df1.index)*10/100),int(len(df1.index)*10/100)+int(len(df1.index)*20/100)),(int(len(df1.index)*10/100)+int(len(df1.index)*20/100),int(len(df1.index)))]
colors = ["red","orange","green"]

#Add trace values to figure accordingly
fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df1[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states', lon = df_sub['Longitude'], lat = df_sub['Latitude'], text= df_sub['text'],
        marker = dict( size = df_sub['Irradiance']*4, color = colors[i], sizemode = 'diameter' ),
        name = names[i]))

# Plot the bubblemap of the USA with Irradiance data
fig.update_layout(
        title_text = '2016 to 2020 US Irradiance<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(scope = 'usa', landcolor = 'rgb(217, 217, 217)')
    )
fig.show()

# **Bubble maps of Average Temperature data:**

In [ ]:
#Bubble map of the world against Avg Temp data:

# Sort values on Avg temp to obtain the top 10%, 10%-20% and 20%+
df1 = df1.sort_values(['Avg temp'],ignore_index=True)
for index,row in df1.iterrows():
  if index+1 <= int(len(df1.index)*10/100):
    df1.loc[index,'colors'] = "10%"
  elif index+1 <= int(len(df1.index)*10/100)+int(len(df1.index)*20/100):
    df1.loc[index,'colors'] = "10-20%"
  else:
    df1.loc[index,'colors'] = "20%+"
df1['Absolute Avg temp'] = abs(df1['Avg temp'])
df1['Absolute Avg temp'] = max(df1['Absolute Avg temp'])+1 - df1['Absolute Avg temp']

# Plot the bubblemap of the natural earth with Avg temp data
fig = px.scatter_geo(df1, lat=df1.Latitude,
                     lon=df1.Longitude,
                     color_discrete_sequence=[ "teal", "steelblue", "lightblue"],
                     color = df1['colors'],
                     hover_data=["City Name", "State Code/Country Name", "Latitude", "Longitude", "Irradiance","Avg temp","Max temp","Min temp","Average NO2"], 
                     size=df1['Absolute Avg temp'],
                     projection="natural earth")
fig.show()

In [ ]:
# Bubble map of the USA against Avg temp data:

# Set values for the sorted Avg temp to obtain the top 10%, 10%-20% and 20%+ and colorcode accordingly
limits = [(0,int(len(df1.index)*10/100)),(int(len(df1.index)*10/100),int(len(df1.index)*10/100)+int(len(df1.index)*20/100)),(int(len(df1.index)*10/100)+int(len(df1.index)*20/100),int(len(df1.index)))]
colors = ["teal", "steelblue", "lightblue"]

#Add trace values to figure accordingly
fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df1[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states', lon = df_sub['Longitude'], lat = df_sub['Latitude'], text= df_sub['text'],
        marker = dict( size = df_sub['Absolute Avg temp'], color = colors[i], sizemode = 'diameter' ),
        name = names[i]))

# Plot the bubblemap of the USA with Avg temp data
fig.update_layout(
        title_text = '2016 to 2020 US Average temperature<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(scope = 'usa', landcolor = 'rgb(217, 217, 217)')
    )
fig.show()

# **Bubble maps of Average NO2 data:**

In [ ]:
#Bubble map of the world against average NO2 data:

# Sort values on Average NO2 to obtain the top 10%, 10%-20% and 20%+
df1 = df1.sort_values(['Average NO2'],ascending=[0],ignore_index=True)
for index,row in df1.iterrows():
  if index+1 <= int(len(df1.index)*10/100):
    df1.loc[index,'colors'] = "10%"
  elif index+1 <= int(len(df1.index)*10/100)+int(len(df1.index)*20/100):
    df1.loc[index,'colors'] = "10-20%"
  else:
    df1.loc[index,'colors'] = "20%+"


# Make all values positive and scaleable
df1['Absolute Average NO2'] = abs(df1['Average NO2'])
df1['Absolute Average NO2'] = df1['Average NO2'] + max(df1['Absolute Average NO2'])+1


# Plot the bubblemap of the natural earth with Average NO2 data
fig = px.scatter_geo(df1, lat=df1.Latitude,
                     lon=df1.Longitude,
                     color_discrete_sequence=[ "Brown", "BurlyWood", "Wheat"],
                     color = df1['colors'],
                     hover_data=["City Name", "State Code/Country Name", "Latitude", "Longitude", "Irradiance","Avg temp","Max temp","Min temp","Average NO2"],  
                     size=df1['Absolute Average NO2'],
                     projection="natural earth")
fig.show()

In [ ]:
# Bubble map of the USA against Average NO2 data:

# Set values for the sorted Average NO2 to obtain the top 10%, 10%-20% and 20%+ and colorcode accordingly
limits = [(0,int(len(df1.index)*10/100)),(int(len(df1.index)*10/100),int(len(df1.index)*10/100)+int(len(df1.index)*20/100)),(int(len(df1.index)*10/100)+int(len(df1.index)*20/100),int(len(df1.index)))]
colors = ["Brown", "BurlyWood", "Wheat"]

#Add trace values to figure accordingly
fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df1[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states', lon = df_sub['Longitude'], lat = df_sub['Latitude'], text= df_sub['text'],
        marker = dict( size = df_sub['Absolute Average NO2']/2, color = colors[i], sizemode = 'diameter' ),
        name = names[i]))

# Plot the bubblemap of the USA with Average NO2 data
fig.update_layout(
        title_text = '2016 to 2020 US Average NO2<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(scope = 'usa', landcolor = 'rgb(217, 217, 217)')
    )
fig.show()

# **Bubble maps of Datacenter Hotspot Index (DHI):**

In [ ]:
# Add Normalized irradiance, Normalized avg temp, Normalized NO2, and DHI columns to the dataframe
df1["Normalized irradiance"] = pd.NaT
df1["Normalized avg temp"] = pd.NaT
df1["Normalized NO2"] = pd.NaT
df1["DHI"] = pd.NaT
a=b=c=1.0

#Calculate and assign the data in the newly added columns of the dataframe
for index, row in df1.iterrows():
  df1.loc[index,"Normalized irradiance"] = df1["Irradiance"][index] / ( max(df1["Irradiance"]) - min(df1["Irradiance"]) )
  df1.loc[index,"Normalized avg temp"] = 1-( (df1["Avg temp"][index] - min(df1["Avg temp"]) ) / ( max(df1["Avg temp"]) - min(df1["Avg temp"]) ))
  df1.loc[index,"Normalized NO2"] = (df1["Average NO2"][index] - min(df1["Average NO2"]) )/ ( max(df1["Average NO2"]) - min(df1["Average NO2"]) )
  df1.loc[index,"DHI"] = ( a * df1["Normalized irradiance"][index] + b * df1["Normalized avg temp"][index] + c * df1["Normalized NO2"][index] ) / 3

#Convert the "object" type columns to numeric to facilitate plotting
df1 = df1.apply(pd.to_numeric, errors='ignore')

#Data to be visible on hover
df1['text'] = 'City Name: '+df1["City Name"]+ \
              '<br>State Code/Country Name: '+df1["State Code/Country Name"]+ \
              '<br>DHI: '+df1["DHI"].astype(str)+ \
              '<br>Irradiance: '+df1["Irradiance"].astype(str)+ \
              '<br>Avg temp: '+df1["Avg temp"].astype(str)+ \
              '<br>Max temp: '+df1["Max temp"].astype(str)+ \
              '<br>Min temp: '+df1["Min temp"].astype(str)+ \
              '<br>Average NO2: '+df1["Average NO2"].astype(str)

In [ ]:
#Bubble map of the world against DHI data:

# Sort values on DHI to obtain the top 10%, 10%-20% and 20%+
df1 = df1.sort_values(['DHI'], ascending=[0],ignore_index=True)
for index,row in df1.iterrows():
  if index+1 <= int(len(df1.index)*10/100):
    df1.loc[index,'colors'] = "10%"
  elif index+1 <= int(len(df1.index)*10/100)+int(len(df1.index)*20/100):
    df1.loc[index,'colors'] = "10-20%"
  else:
    df1.loc[index,'colors'] = "20%+"

# Plot the bubblemap of the natural earth with DHI data
fig = px.scatter_geo(df1, lat=df1.Latitude,
                     lon=df1.Longitude,
                     color_discrete_sequence=[ "darkgreen", "limegreen", "yellowgreen"],
                     color = df1['colors'],
                     hover_data=["City Name", "State Code/Country Name", "Irradiance","Avg temp","Max temp","Min temp","Average NO2"], 
                     size=df1['DHI'],
                     projection="natural earth")
fig.show()

In [ ]:
# Bubble map of the USA against DHI data:
              
# Set values for the sorted DHI to obtain the top 10%, 10%-20% and 20%+ and colorcode accordingly
limits = [(0,int(len(df1.index)*10/100)),(int(len(df1.index)*10/100),int(len(df1.index)*10/100)+int(len(df1.index)*20/100)),(int(len(df1.index)*10/100)+int(len(df1.index)*20/100),int(len(df1.index)))]
colors = ["darkgreen", "limegreen", "yellowgreen"]

#Add trace values to figure accordingly
fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df1[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states', lon = df_sub['Longitude'], lat = df_sub['Latitude'], text= df_sub['text'],
        marker = dict( size = df_sub['DHI']*30, color = colors[i], sizemode = 'diameter' ),
        name = names[i]))

# Plot the bubblemap of the USA with DHI data
fig.update_layout(
        title_text = '2016 to 2020 US Datacenter Hotspot Index<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(scope = 'usa', landcolor = 'rgb(217, 217, 217)')
    )
fig.show()

# **Dendogram and Kmeans**

In [ ]:
dfitn=df_irradiance_temp_NO2
dfitn = dfitn.filter(items=['City Name','State Code/Country Name','Irradiance', 'Avg temp','Average NO2'])

dfitn = dfitn.set_index(['City Name','State Code/Country Name'])
dfitn = dfitn.apply(pd.to_numeric, errors='ignore')
Ndf=(dfitn-dfitn.min())/(dfitn.max()-dfitn.min())
Ndf=Ndf.reset_index()

In [ ]:
Ndf = Ndf.set_index(['City Name','State Code/Country Name'])
Z = linkage(Ndf, 'ward',metric='euclidean')
plt.figure(figsize=(26,12))
dendrogram(Z,
           orientation='top',
           distance_sort='descending',
           show_leaf_counts=True,
           labels = Ndf.index,
           leaf_font_size=12)
plt.show()

In [ ]:
#Using fcluster add cluster count as 2 to make 2 groups for data
clusterCount = 2
clusters_agg = fcluster(Z, t=clusterCount, criterion='maxclust')
# fluster method assigns cluster index to data categorizing into 2 values.
Ndf["Cluster index"] = clusters_agg
# Changing the cluster index values to 0,1 from 1,2.
Ndf.loc[Ndf["Cluster index"] == 1, "Cluster index"] = 0
Ndf.loc[Ndf["Cluster index"] == 2, "Cluster index"] = 1

Ndf=Ndf.reset_index()
Ndf

In [ ]:
#3d scatter plot showing the clusters in 3d space.
scatter3dPlot = px.scatter_3d(Ndf, x='Irradiance', y='Avg temp', z='Average NO2',
              color='Cluster index', hover_data=['City Name','State Code/Country Name',"Irradiance","Avg temp","Average NO2"])
# to remove the legend since the values are supposed to be 0,1.
scatter3dPlot.update(layout_coloraxis_showscale=False)
scatter3dPlot.show()

In [ ]:
# 2d scatter matrix using plotly express
ScatterPlot1 = px.scatter_matrix(Ndf,
    dimensions=["Irradiance", "Avg temp", "Average NO2"],
    color="Cluster index", hover_data=['City Name','State Code/Country Name',"Irradiance","Avg temp","Average NO2"])
# removing the legend as color distinction shows the clusters.
ScatterPlot1.update(layout_coloraxis_showscale=False)
ScatterPlot1.show()

In [ ]:
# Pandas scatter matrix to show the cluster patters with 6 clusters.
X = Ndf[['Irradiance','Avg temp','Average NO2']];
y = Ndf['Cluster index'];
# removing irregular y axis values and plotting the scatter matix
axes=pd.plotting.scatter_matrix(X,c=y,range_padding=0,alpha=2, figsize=(16,12))
axes[0,0].set_yticklabels([0,0.2,0.4,0.6,0.8,1])

In [ ]:
# Considering 6 clusters and normalizing the data
clusterCount = 6
clusters_agg2 = fcluster(Z, t=clusterCount, criterion='maxclust')
normalized_df2 = Ndf.copy()
normalized_df2["Cluster index"] = clusters_agg2
Ndf.loc[Ndf["Cluster index"] == 6, "Cluster index"] = 0

In [ ]:
#3d scatter plot showing the clusters in 3d space.
scatter3dPlot = px.scatter_3d(normalized_df2, x='Irradiance', y='Avg temp', z='Average NO2',
              color='Cluster index', hover_data=['City Name','State Code/Country Name',"Irradiance","Avg temp","Average NO2"])

scatter3dPlot.update(layout_coloraxis_showscale=False)
scatter3dPlot.show()

In [ ]:
# Scatter plot showing 6 clusters
ScatterPlot2 = px.scatter_matrix(normalized_df2,
    dimensions=["Irradiance", "Avg temp", "Average NO2"],
    color="Cluster index", hover_data=["Irradiance","Avg temp","Average NO2",'City Name','State Code/Country Name'])
ScatterPlot2.update(layout_coloraxis_showscale=False)
ScatterPlot2.show()

In [ ]:
X = normalized_df2[['Irradiance','Avg temp','Average NO2']];
y = normalized_df2['Cluster index'];
axes2=pd.plotting.scatter_matrix(X,c=y, range_padding=0, alpha=2, figsize=(16,12))
axes2[0,0].set_yticklabels([0,0.2,0.4,0.6,0.8,1])

In [ ]:
k_count=2
kmenas = normalized_df2.filter(items=['Irradiance','Avg temp','Average NO2'])
kviz,label = kmeans2(kmenas, k_count, iter=10, minit='++', missing='warn', check_finite=True)

In [ ]:
normalized_df3 = Ndf.copy()
normalized_df3["Cluster index"] = label
normalized_df3=normalized_df3.reset_index()
normalized_df3

In [ ]:
#3d scatter plot of data obtained from k-means analysis
scatter3dPlot3 = px.scatter_3d(normalized_df3, x='Irradiance', y='Avg temp', z='Average NO2',
              color='Cluster index', hover_data=['City Name','State Code/Country Name',"Irradiance","Avg temp","Average NO2"])
# to remove the legend since the values are supposed to be 0,1.
scatter3dPlot3.update(layout_coloraxis_showscale=False)
scatter3dPlot3.show()

In [ ]:
# 2d scatter matrix using plotly express for data obtained from kmeans analysis 
ScatterPlot3 = px.scatter_matrix(normalized_df3,
    dimensions=["Irradiance", "Avg temp", "Average NO2"],
    color="Cluster index", hover_data=['City Name','State Code/Country Name',"Irradiance","Avg temp","Average NO2"])
# removing the legend as color distinction shows the clusters.
ScatterPlot3.update(layout_coloraxis_showscale=False)
ScatterPlot3.show()

In [ ]:
# Pandas scatter matrix to show the cluster patters with 2 clusters.
X = normalized_df3[['Irradiance','Avg temp','Average NO2']];
y = normalized_df3['Cluster index'];
# removing irregular y axis values and plotting the scatter matix
axes2=pd.plotting.scatter_matrix(X,c=y,range_padding=0,alpha=2, figsize=(16,12))
axes2[0,0].set_yticklabels([0,0.2,0.4,0.6,0.8,1])

# **Pareto comparison**

In [ ]:
df_final=df_irradiance_temp_NO2
df_final['DominationCount']=0;

In [ ]:
for i, row in df_final.iterrows():
    irr1 = row['Irradiance'] # Get the values irr, temp and no for this row
    temp1 = row['Avg temp']
    no1 = row['Average NO2']
    dominance = 0
    for j, row2 in df_final.iterrows(): # Go over the complete dataset
        irr2 = row2['Irradiance']
        temp2 = row2['Avg temp']
        no2 = row2['Average NO2']
        if (no2 < no1) or (temp2 > temp1) or (irr2 < irr1): # Check if the row has all features >= the current row
          continue
        if (no2 > no1) or (temp2 < temp1) or (irr2 > irr1): # And one of the features dominates
          dominance += 1
    
    df_final.at[i, 'DominationCount'] = dominance # Output the final dominance
df_final=df_final.set_index('City Name')

Pareto Analysis on Irradiance

In [ ]:
df_final = df_final.sort_values(by='DominationCount', ascending=True)
df_final['Irradiancecum'] = df_final['Irradiance'].cumsum()/df_final['Irradiance'].sum()*100

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

color1 = 'steelblue'
color2 = 'red'
line_size = 4
fig, ax = plt.subplots(figsize=(20,10))
ax.bar(df_final.index, df_final['Irradiance'], color=color1)
ax2 = ax.twinx()
ax2.plot(df_final.index, df_final['Irradiancecum'], color=color2, marker="D", ms=line_size)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax.tick_params(axis='y', colors=color1,)
ax2.tick_params(axis='y', colors=color2)

ax.set_xticklabels(df_final.index, rotation = 90)
plt.title("Irradiance Distribution")
ax.set_ylabel('Irradiance(kW-hr/m^2/day)')
ax2.set_ylabel('Cumulative Distribution')

plt.show()

Pareto Analysis for Average Temperature

In [ ]:
df_final = df_final.sort_values(by='Avg temp', ascending=False)
df_final['Avgtempcum'] = df_final['Avg temp'].cumsum()/df_final['Avg temp'].sum()*100

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

color1 = 'steelblue'
color2 = 'red'
line_size = 4
fig, ax = plt.subplots(figsize=(20,10))
ax.bar(df_final.index, df_final['Avg temp'], color=color1)
ax2 = ax.twinx()
ax2.plot(df_final.index, df_final['Avgtempcum'], color=color2, marker="D", ms=line_size)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax.tick_params(axis='y', colors=color1)
ax2.tick_params(axis='y', colors=color2)

ax.set_xticklabels(df_final.index, rotation = 90)
plt.title("Temperature Distribution")
ax.set_ylabel('Average Temperature(C)')
ax2.set_ylabel('Cumulative Distribution')
plt.show()

Pareto Analysis for Average No2

In [ ]:
df_final = df_final.sort_values(by='Average NO2', ascending=False)
df_final['AvgNo2cum'] = df_final['Average NO2'].cumsum()/df_final['Average NO2'].sum()*100

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

color1 = 'steelblue'
color2 = 'red'
line_size = 4
fig, ax = plt.subplots(figsize=(20,10))
ax.bar(df_final.index, df_final['Average NO2'], color=color1)
ax2 = ax.twinx()
ax2.plot(df_final.index, df_final['AvgNo2cum'], color=color2, marker="D", ms=line_size)
ax2.yaxis.set_major_formatter(PercentFormatter())

ax.tick_params(axis='y', colors=color1)
ax2.tick_params(axis='y', colors=color2)

ax.set_xticklabels(df_final.index, rotation = 90)
plt.title("NO2()")
ax.set_ylabel('Average Temperature(C)')
ax2.set_ylabel('Cumulative Distribution')
plt.show()

Plots

3-D SCATTER CHART

In [ ]:
import plotly.express as px

df_final_irr=df_final.copy()
is_0=df_final['DominationCount']==0
fig = px.scatter_3d(df_final_irr, x='Irradiance', y='Average NO2', z='Avg temp',
                    color='DominationCount',symbol=is_0,hover_data=[df_final_irr.index,'DominationCount'])
fig.update(layout_showlegend=False)

fig.show()

SCATTTER MATRIX CHARTS

In [ ]:
import plotly.express as px

df_final2=df_final.sort_index()
fig = px.scatter_matrix(df_final2,
    dimensions=["Irradiance", "Average NO2", "Avg temp"],
    color=df_final2.index)

fig.show()

In [ ]:
df1=df_final.reset_index(inplace=False)

In [ ]:
df1=df1.drop(columns=['DominationCount'])

df2 = pd.melt(df1, id_vars=['City Name','Irradiancecum','Avgtempcum','AvgNo2cum'], 
                  var_name="Indicators", value_name="Value")

POLAR SCATTER CHART

In [ ]:
import plotly.express as px

df_final3=df2.sort_values('City Name')

fig = px.scatter_polar(df_final3,r='Value'
                       ,theta='Indicators',color='City Name'
                       )
fig.show()

POLAR LINE CHART

In [ ]:
import plotly.express as px

fig = px.line_polar(df_final3,r='Value'
                       ,theta='Indicators',color='City Name',line_close=True,
                     template="plotly_dark"
                       )
fig.show()